In [1]:
import os
import json
import time
import openai

In [2]:
openai.api_key = os.getenv('OPENAI_API_KEY')

## Fine-tuning

In [3]:
jsonl_file_path = "prompts2.jsonl"

In [4]:
def upload_training_file(file_path):
    try:
        with open(file_path, "rb") as f:
            response = openai.files.create(file=f, purpose="fine-tune")
            print(f"File uploaded successfully. File ID: {response.id}")
            return response.id
    except Exception as e:
        print(f"File upload failed: {e}")
        return None

training_file_id = upload_training_file(jsonl_file_path)

File uploaded successfully. File ID: file-Cc9ZPhXYq8cRd5Mc3Xhvp3


In [5]:
def fine_tune_model(training_file_id, base_model="gpt-4o-2024-08-06"):
    try:
        fine_tune_response = openai.fine_tuning.jobs.create(
            training_file=training_file_id,
            model=base_model
        )
        print(f"Fine-tuning job started. Job ID: {fine_tune_response.id}")
        return fine_tune_response.id
    except Exception as e:
        print(f"Failed to start fine-tuning: {e}")
        return None

fine_tune_job_id = fine_tune_model(training_file_id)

Fine-tuning job started. Job ID: ftjob-nYOGsinYa8SVxiB3hiApyK6A


In [6]:
def monitor_fine_tuning_job(job_id):
    try:
        while True:
            response = openai.fine_tuning.jobs.retrieve(job_id)
            print(f"Job Status: {response.status}")
            
            if response.status in ['succeeded', 'failed']:
                print(f"Fine-tuning job completed with status: {response.status}")
                
                if response.status == 'failed':
                    print(f"Fetching error details for job: {job_id}")
                    events = openai.fine_tuning.jobs.list_events(fine_tuning_job_id=job_id)
                    for event in events.data:
                        print(f"Event: {event.message}")  
                        
                if response.status == 'succeeded':
                    print(f"Fine-tuned model: {response.fine_tuned_model}")
                    
                return response.fine_tuned_model
            time.sleep(10)  
    except Exception as e:
        print(f"Error monitoring fine-tuning job: {e}")

fine_tuned_model_id = monitor_fine_tuning_job(fine_tune_job_id)

Job Status: validating_files
Job Status: validating_files
Job Status: validating_files
Job Status: validating_files
Job Status: validating_files
Job Status: validating_files
Job Status: running
Job Status: running
Job Status: running
Job Status: running
Job Status: running
Job Status: running
Job Status: running
Job Status: running
Job Status: running
Job Status: running
Job Status: running
Job Status: running
Job Status: running
Job Status: running
Job Status: running
Job Status: running
Job Status: running
Job Status: running
Job Status: running
Job Status: running
Job Status: running
Job Status: running
Job Status: running
Job Status: running
Job Status: running
Job Status: running
Job Status: running
Job Status: running
Job Status: running
Job Status: running
Job Status: running
Job Status: running
Job Status: running
Job Status: running
Job Status: running
Job Status: running
Job Status: running
Job Status: running
Job Status: running
Job Status: running
Job Status: running
Job St

## Using your fine-tuned model

In [10]:
def use_fine_tuned_model(model_id, prompt):
    try:
        response = openai.chat.completions.create(
            model=model_id,
            messages=[
                {"role": "system", "content": "You are a game master."},
                {"role": "user", "content": prompt}
            ]
        )
        return response.choices[0].message.content.strip()
    except Exception as e:
        print(f"Error using the fine-tuned model: {e}")
        return None

fine_tuned_model_id = 'ft:gpt-4o-2024-08-06:personal::ArlwY1XZ'

prompt = '''
    You are given a dark story, which description is known to the user 
    however its solution is known only to you. User tried to solve the puzzle. 
    Your goal is to decide wheter the user found the essence of the solution. 
    Respond with TRUE, if the user found out the essence of the puzzle 
    or FALSE if he failed to do so. The known description to the dark story: 
    Man goes to the bar and asks for a glass of water. Barman takes a gun and puts it 
    to the man's head. The man says 'thank you' and leaves the bar. 
    and here is solution to the dark story: The man had hiccups, he wanted to end them with a glass of water, 
    but instead, the barman scared him which ended the hiccups. 
    user solution is as follow: man was mentaly ill.
'''


response = use_fine_tuned_model(fine_tuned_model_id, prompt)
    
if response:
    print(f"Model Response:\n{response}")
else:
    print("Failed to get a response from the fine-tuned model.")

Model Response:
FALSE
